## Extract functional MRI data

The goal is to extract the fmri data (that were recorded during movie viewing) via a masker and save them to disk.

Note: Due to the large size of the dataset, this is done from a harddisk (after downloading the data from openNeuro). Thus, this step is documented, but cannot be exectuted unless you download and preprocess the data exactly as indicated.

#### import modules

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sys, os, nilearn, shutil, glob
from nilearn import input_data, datasets
from nilearn import datasets, plotting
from nilearn.image import new_img_like, load_img, get_data
from nilearn.interfaces.fmriprep import load_confounds
from nilearn import input_data, datasets

#### set up the brain masker

In [ ]:
shen_dictionary   = np.load('/Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/partlycloudy_twins/data/02_rois_masks_maps/shen_dictionary.npy', allow_pickle = True).item()
shen_file         = '/Users/ralfschmaelzle/Documents//GITHUB/nomcomm/PAPERS_WORKING/partlycloudy_twins/data/02_rois_masks_maps/shen_2mm_268_parcellation.nii'
path_dir          = "/Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/partlycloudy_twins/data/03_extract_ts/"

all293_rois_file  = '/Users/ralfschmaelzle/Documents/GITHUB/nomcomm/PAPERS_WORKING/partlycloudy_twins/data/02_rois_masks_maps/allcombined.nii'

all293_masker     = input_data.NiftiLabelsMasker(all293_rois_file, 
                                             detrend     = True, 
                                             standardize = True,
                                             high_pass   = 0.01, 
                                             t_r         = 0.8, 
                                             low_pass    = 0.12 );

all293_masker.fit('/Volumes/Backup Plus/twins_fmriprep/sub-0001/ses-02/func/sub-0001_ses-02_task-partlycloudy_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')

#### set up files

In [ ]:
mylist = [f for f in glob.glob("/Volumes/Backup Plus/twins_fmriprep/sub*/ses-02/func/*_ses-02_task-partlycloudy_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz")]
print(len(mylist)) #[-84:65]
mylist[:3]

In [ ]:
for curr_file in mylist:

    confounds_simple, sample_mask = load_confounds(
                                     curr_file,    
                                     strategy=["motion", "wm_csf"],
                                     motion="basic", 
                                     wm_csf="basic")

    curr_out_file = curr_file[-84:65]
    curr_out_file_path = path_dir + curr_out_file + '_LTR_Z_THP001_LP012_shen268_pauli16_aan9.npy'
    print(curr_out_file_path)

    if not os.path.exists(curr_out_file_path):
            time_series = all293_masker.fit_transform(curr_file,confounds=confounds_simple)
        
            np.save(curr_out_file_path, time_series) 